# Demonstration from pinecone webiner:

Source code: https://colab.research.google.com/drive/1ZbXzzMFyMQ-nQf-gQXWAv4IRDI90rXpa?usp=sharing&_hsmi=255277680#scrollTo=8Br4lPWNmZYy

In [1]:
!pip install -qU datasets pinecone-client[grpc] langchain openai tqdm

In [2]:
import os
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
PINECONE_ENV = "us-east4-gcp"

In [3]:
from datasets import load_dataset

data = load_dataset(
    'jamescalam/lex-transcripts',
    split='train'
)
data

Found cached dataset json (C:/Users/Masa/.cache/huggingface/datasets/jamescalam___json/jamescalam--lex-transcripts-6a9688b7915283fe/0.0.0/fe5dd6ea2639a6df622901539cb550cf8797e5a6b2dd7af1cf934bed8e233e6e)


Dataset({
    features: ['video_id', 'channel_id', 'title', 'published', 'transcript', 'source'],
    num_rows: 499
})

In [4]:
data[0]

{'video_id': '_SpptYg_0Rs',
 'channel_id': 'UCSHZKyawb77ixDdsGog4iWA',
 'title': 'Are We Living in a Simulation? with George Hotz and Lex Fridman | AI Podcast Clips',
 'published': datetime.datetime(2019, 8, 29, 13, 9, 2),
 'transcript': " Do you think we're living in a simulation? Yes, but it may be unfalsifiable. What do you mean by unfalsifiable? So if the simulation is designed in such a way that they did like a formal proof to show that no information can get in and out, and if their hardware is designed for anything in the simulation to always keep the hardware in spec, it may be impossible to prove whether we're in a simulation or not. So they've designed it such that it's a closed system, you can't get outside the system? Well maybe it's one of three worlds. We're either in a simulation which can be exploited, we're in a simulation which not only can't be exploited, but like the same thing is true about VMs. A really well designed VM, you can't even detect if you're in a VM or 

In [5]:
import tiktoken

tokenizer = tiktoken.get_encoding('cl100k_base')  # cl100k base is encoder used by ada-002

# define a length function
def tiktoken_len(text: str) -> int:
    tokens = tokenizer.encode(text, disallowed_special=())
    return len(tokens)

In [6]:
tiktoken_len("here is a random sentence we can get the token length for")

12

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,  # number of tokens overlap between chunks
    length_function=tiktoken_len,  # token count function
    separators=['\n\n', '.\n', '\n', '.', '?', '!', ' ', '']
)

In [8]:
from tqdm.auto import tqdm

new_data = []

for row in tqdm(data):
    chunks = text_splitter.split_text(row['transcript'])
    row.pop('transcript')
    for i, text in enumerate(chunks):
        new_data.append({**row, **{'chunk': i, 'text': text}})

  0%|          | 0/499 [00:00<?, ?it/s]

In [9]:
new_data[0]

{'video_id': '_SpptYg_0Rs',
 'channel_id': 'UCSHZKyawb77ixDdsGog4iWA',
 'title': 'Are We Living in a Simulation? with George Hotz and Lex Fridman | AI Podcast Clips',
 'published': datetime.datetime(2019, 8, 29, 13, 9, 2),
 'source': 'https://youtu.be/_SpptYg_0Rs',
 'chunk': 0,
 'text': "Do you think we're living in a simulation? Yes, but it may be unfalsifiable. What do you mean by unfalsifiable? So if the simulation is designed in such a way that they did like a formal proof to show that no information can get in and out, and if their hardware is designed for anything in the simulation to always keep the hardware in spec, it may be impossible to prove whether we're in a simulation or not. So they've designed it such that it's a closed system, you can't get outside the system? Well maybe it's one of three worlds. We're either in a simulation which can be exploited, we're in a simulation which not only can't be exploited, but like the same thing is true about VMs. A really well designe

In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [11]:
import pinecone

pinecone.init(
    api_key=PINECONE_API_KEY,  # app.pinecone.io
    environment=PINECONE_ENV  # next to API key in console
)

index_name = "lex-agent"

if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        index_name, dimension=1536
    )

index = pinecone.Index(index_name)

In [12]:
batch_size = 100

for i in tqdm(range(0, len(new_data), batch_size)):
    # get end of batch
    i_end = min(len(new_data), i+batch_size)
    # get batch of records
    metadatas = new_data[i:i_end]
    ids = [f"{meta['video_id']}-{meta['chunk']}" for meta in metadatas]
    texts = [meta['text'] for meta in metadatas]
    xc = embeddings.embed_documents(texts)
    to_upsert = zip(ids, xc, metadatas)
    # now add to Pinecone vec DB
    index.upsert(vectors=to_upsert)

  0%|          | 0/126 [00:00<?, ?it/s]

In [13]:
from langchain.vectorstores import Pinecone

vectordb = Pinecone(
    index=index,
    embedding_function=embeddings.embed_query,
    text_key="text"
)

In [14]:
from langchain.chat_models import ChatOpenAI

llm=ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [32]:
from langchain.chains import RetrievalQA

retriever = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectordb.as_retriever(), 
)

In [33]:
tool_desc = """Use this tool to answer user questions using Lex
Fridman podcasts. If the user states 'ask Lex' use this tool to get
the answer. This tool can also be used for follow up questions from
the user."""

In [34]:
from langchain.agents import Tool

tools = [Tool(
    func=retriever.run,
    description=tool_desc,
    name='Lex Fridman DB'
)]

In [35]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(
    memory_key="chat_history",  # important to align with agent prompt (below)
    k=5,
    return_messages=True
)

In [36]:
from langchain.agents import initialize_agent

conversational_agent = initialize_agent(
    agent='chat-conversational-react-description', 
    tools=tools, 
    llm=llm,
    verbose=True,
    max_iterations=2,
    early_stopping_method="generate",
    memory=memory,
)

In [37]:
conversational_agent.agent.llm_chain.prompt

ChatPromptTemplate(input_variables=['input', 'chat_history', 'agent_scratchpad'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template='Assistant is a large language model trained by OpenAI.\n\nAssistant is designed to be able to assist with a wide range of tasks, from answering simple questions to providing in-depth explanations and discussions on a wide range of topics. As a language model, Assistant is able to generate human-like text based on the input it receives, allowing it to engage in natural-sounding conversations and provide responses that are coherent and relevant to the topic at hand.\n\nAssistant is constantly learning and improving, and its capabilities are constantly evolving. It is able to process and understand large amounts of text, and can use this knowledge to provide accurate and informative responses to a wide range of questions. Additionally, A

In [38]:
sys_msg = """You are a helpful chatbot that answers the user's questions.
"""

prompt = conversational_agent.agent.create_prompt(
    system_message=sys_msg,
    tools=tools
)
conversational_agent.agent.llm_chain.prompt = prompt

In [39]:
conversational_agent.agent.llm_chain.prompt

ChatPromptTemplate(input_variables=['input', 'chat_history', 'agent_scratchpad'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template="You are a helpful chatbot that answers the user's questions.\n", template_format='f-string', validate_template=True), additional_kwargs={}), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='TOOLS\n------\nAssistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:\n\n> Lex Fridman DB: Use this tool to answer user questions using Lex\nFridman podcasts. If the user states \'ask Lex\' use this tool to get\nthe answer. This tool can also be used for follow up questions from\nthe user.\n\nRESPONSE FORMAT INSTRUCTION

In [40]:
conversational_agent.agent.llm_chain.prompt.input_variables

['input', 'chat_history', 'agent_scratchpad']

In [41]:
conversational_agent.agent.llm_chain.prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template="You are a helpful chatbot that answers the user's questions.\n", template_format='f-string', validate_template=True), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history'),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='TOOLS\n------\nAssistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:\n\n> Lex Fridman DB: Use this tool to answer user questions using Lex\nFridman podcasts. If the user states \'ask Lex\' use this tool to get\nthe answer. This tool can also be used for follow up questions from\nthe user.\n\nRESPONSE FORMAT INSTRUCTIONS\n----------------------------\n\nWhen responding to me, please output a response in one of two formats:\n\n**Option 1:**\nUse thi

In [42]:
conversational_agent.agent.llm_chain.prompt.messages[0]

SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], output_parser=None, partial_variables={}, template="You are a helpful chatbot that answers the user's questions.\n", template_format='f-string', validate_template=True), additional_kwargs={})

In [43]:
print(
    conversational_agent.agent.llm_chain.prompt.messages[0].prompt.template
)

You are a helpful chatbot that answers the user's questions.



In [44]:
conversational_agent.agent.llm_chain.prompt.messages[1]

MessagesPlaceholder(variable_name='chat_history')

In [45]:
conversational_agent.agent.llm_chain.prompt.messages[2]

HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], output_parser=None, partial_variables={}, template='TOOLS\n------\nAssistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:\n\n> Lex Fridman DB: Use this tool to answer user questions using Lex\nFridman podcasts. If the user states \'ask Lex\' use this tool to get\nthe answer. This tool can also be used for follow up questions from\nthe user.\n\nRESPONSE FORMAT INSTRUCTIONS\n----------------------------\n\nWhen responding to me, please output a response in one of two formats:\n\n**Option 1:**\nUse this if you want the human to use a tool.\nMarkdown code snippet formatted in the following schema:\n\n```json\n{{\n    "action": string \\ The action to take. Must be one of Lex Fridman DB\n    "action_input": string \\ The input to the action\n}}\n```\n\n**Option #2:**\nUse this if you want to respond directly to the

In [46]:
print(
    conversational_agent.agent.llm_chain.prompt.messages[2].prompt.template
)

TOOLS
------
Assistant can ask the user to use tools to look up information that may be helpful in answering the users original question. The tools the human can use are:

> Lex Fridman DB: Use this tool to answer user questions using Lex
Fridman podcasts. If the user states 'ask Lex' use this tool to get
the answer. This tool can also be used for follow up questions from
the user.

RESPONSE FORMAT INSTRUCTIONS
----------------------------

When responding to me, please output a response in one of two formats:

**Option 1:**
Use this if you want the human to use a tool.
Markdown code snippet formatted in the following schema:

```json
{{
    "action": string \ The action to take. Must be one of Lex Fridman DB
    "action_input": string \ The input to the action
}}
```

**Option #2:**
Use this if you want to respond directly to the human. Markdown code snippet formatted in the following schema:

```json
{{
    "action": "Final Answer",
    "action_input": string \ You should put what yo

In [47]:
conversational_agent("hi how are you")



> Entering new AgentExecutor chain...
{
    "action": "Final Answer",
    "action_input": "I'm just a chatbot, so I don't have feelings, but I'm here to help you with any questions you have!"
}

> Finished chain.


{'input': 'hi how are you',
 'chat_history': [],
 'output': "I'm just a chatbot, so I don't have feelings, but I'm here to help you with any questions you have!"}

In [48]:
conversational_agent("ask lex about the future of ai")



> Entering new AgentExecutor chain...
{
    "action": "Lex Fridman DB",
    "action_input": "What did Lex Fridman say about the future of AI?"
}
Observation: Lex Fridman discussed the potential for AI to become companions and proliferate in the digital space, but he also emphasized the importance of democratizing AI and preventing large tech companies from using it to control human beings. He also expressed his belief that AI will not become our masters, but rather, humans will become the masters of AI. Additionally, he discussed the concern that automation and powerful AI will lead to job loss, but he believes that ultimately, technology will enhance human intelligence and create new job opportunities. Finally, he highlighted the theme of his upcoming book, Singularity's Nearer, which argues that things are getting better in all aspects of life.
Thought:{
    "action": "Final Answer",
    "action_input": "Based on Lex Fridman's discussions, he believes that AI will not become our ma

{'input': 'ask lex about the future of ai',
 'chat_history': [HumanMessage(content='hi how are you', additional_kwargs={}),
  AIMessage(content="I'm just a chatbot, so I don't have feelings, but I'm here to help you with any questions you have!", additional_kwargs={})],
 'output': "Based on Lex Fridman's discussions, he believes that AI will not become our masters, but rather, humans will become the masters of AI. He also believes that technology will enhance human intelligence and create new job opportunities. Finally, he argues that things are getting better in all aspects of life in his upcoming book, Singularity's Nearer."}

In [ ]:
conversational_agent("what does he think about space exploration?")



> Entering new AgentExecutor chain...
{
    "action": "Lex Fridman DB",
    "action_input": "What does Lex Fridman think about space exploration?"
}
Observation: Lex Fridman is a strong advocate for space exploration and believes that it is important for humanity to expand beyond Earth. He has had several guests on his podcast who share this view, including Eric Weinstein, who also believes that getting off this planet is crucial for the survival and progress of humanity. Lex has also expressed interest in the potential for extraterrestrial life and the possibility of visiting other worlds.
Thought: